In [1]:
!pip install --quiet /kaggle/input/kerasapplications
!pip install --quiet /kaggle/input/efficientnet-git

In [2]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential
import cv2
from tensorflow.keras.layers import Dense
import os
import efficientnet.tfkeras as efn
import numpy as np

# Loading in model

In [3]:
IMG_SIZE = 256

In [4]:
def make_model():
    model = Sequential()
    model.add(efn.EfficientNetB4(include_top = False,
                                  weights = None,
                                  pooling = 'avg',
                                  input_shape = (IMG_SIZE, IMG_SIZE, 3)))
    model.add(Dense(5, activation = 'softmax'))
    model.layers[0].trainable = True
    
#     optimizer = Adam(lr = 0.01)
#     #tpu_optimizer = tf.experimental.contrib.tpu.CrossShardOptimizer(optimizer)
#     model.compile(optimizer=optimizer, 
#                   loss='categorical_crossentropy', 
#                   metrics=['accuracy'])
    return model

In [5]:
model = make_model()
model.load_weights('../input/v0-1-cassava-efficientnet-training/EnetB4_top_trainable/best_model.hdf5')

# Looking at the format of submission

In [6]:
pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')

,image_id,label
0,2216849948.jpg,4


# Making Predictions - 

In [7]:
sub_dict = {'image_id':[], 'label':[]}

In [8]:
test_data_path = '../input/cassava-leaf-disease-classification/test_images/'
for img_path in os.listdir(test_data_path):
    img = cv2.imread(test_data_path + img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    prediction = np.argmax(model.predict([img.reshape(-1, IMG_SIZE, IMG_SIZE, 3)]))
    sub_dict['image_id'].append(img_path)
    sub_dict['label'].append(prediction)

In [9]:
submission_df = pd.DataFrame(sub_dict)
submission_df.head()

,image_id,label
0,2216849948.jpg,0


In [10]:
submission_df.to_csv("submission.csv", index=False)